[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/)

# Aula 2 — Introdução ao `pandas` para Epidemiologia

> **Curso:** Python para Epidemiologistas — Aula 2  
> **Foco:** criação/inspeção de DataFrames, seleção e filtros (`query`/máscaras), criação de colunas (`assign`) e resumo com `groupby().agg()`; mini‑EDA (Exploratory Data Analysis) com dados sintéticos + **exemplo** de leitura de dados do **OpenDataSUS (SIVEP‑Gripe)**.

## Objetivos de aprendizagem
- Criar e inspecionar `DataFrame`s (`shape`, `info`, `describe`, `value_counts`).
- Selecionar colunas, filtrar linhas com **máscaras booleanas** e com `DataFrame.query`.
- Criar/transformar variáveis com `assign` e resumir com `groupby().agg`.
- Realizar mini‑exploração de um *line list* epidemiológico (frequências por **região** e **idade**).
- Carregar dados **remotos** do **OpenDataSUS (SIVEP‑Gripe)** *com fallback* para dados sintéticos.

## Pré‑requisitos
- Aula 1 (Fundamentos): tipos, estruturas, condicionais/loops, funções, list comprehensions.

## Como usar no Colab
1. Abra no Colab e rode **Runtime → Run all** para executar tudo do zero.
2. Salve sua cópia: **File → Save a copy in Drive**.
3. Para reiniciar: **Runtime → Restart runtime**.


In [ ]:
#@title Instalar/Carregar dependências
# Usaremos bibliotecas já disponíveis no Colab.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io, requests

pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 120)

# Semente para reprodutibilidade
rng = np.random.default_rng(42)
print("Ambiente carregado.")

## 1) Dados: sintético + exemplo OpenDataSUS (fallback robusto)

Trabalharemos com um **line list** sintético e tentaremos, como **exemplo**, ler uma amostra do **OpenDataSUS (SIVEP‑Gripe)**.
Se a leitura remota falhar (rede/permits), seguimos apenas com o **sintético**.

### Dicionário de variáveis (sintético)
- `id`: identificador do caso.
- `age`: idade em anos.
- `sex`: `'F'`/`'M'`.
- `region`: macrorregião (`Norte`, `Nordeste`, `Sudeste`, `Sul`, `Centro-Oeste`).
- `onset_date`: data de início dos sintomas.
- `report_date`: data de notificação.
- `hospitalized`: `'internado'`/`'nao_internado'`.

> No **SIVEP‑Gripe**, nomes de colunas podem diferir (ex.: `CS_SEXO`, `NU_IDADE_N`). A função abaixo tenta **normalizar** para `sex`, `age`, `onset_date`, `report_date`, `region`, `hospitalized`.


In [ ]:
# Funções utilitárias
from datetime import datetime

def generate_synthetic_line_list(n=1200, seed=42):
    rng = np.random.default_rng(seed)
    ids = np.arange(1, n+1)
    ages = rng.integers(0, 90, size=n)
    sex = rng.choice(['F', 'M'], size=n, p=[0.53, 0.47])
    regions = rng.choice(['Norte', 'Nordeste', 'Sudeste', 'Sul', 'Centro-Oeste'], size=n, p=[0.08,0.28,0.43,0.14,0.07])
    start = np.datetime64('2023-01-01')
    onset = start + rng.integers(0, 300, size=n).astype('timedelta64[D]')
    report_delay = rng.integers(0, 10, size=n).astype('timedelta64[D]')
    report = onset + report_delay
    hosp = rng.choice(['internado', 'nao_internado'], size=n, p=[0.18, 0.82])
    return pd.DataFrame({
        'id': ids,
        'age': ages,
        'sex': sex,
        'region': regions,
        'onset_date': pd.to_datetime(onset),
        'report_date': pd.to_datetime(report),
        'hospitalized': hosp
    })

def try_load_opendatasus_srag(sample_n=20000):
    """Tenta ler amostra do SIVEP‑Gripe (OpenDataSUS).
    Usa requests com cabeçalho User-Agent para reduzir 403.
    Retorna DataFrame normalizado OU None em falha.
    """
    url = 'https://s3.sa-east-1.amazonaws.com/ckan.saude.gov.br/SRAG/2023/INFLUD23-26-06-2025.csv'
    try:
        resp = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}, timeout=60)
        resp.raise_for_status()
        # print(resp.content)
        df_raw = pd.read_csv(io.BytesIO(resp.content), sep=';', encoding='latin1', nrows=sample_n, low_memory=False)

        if df_raw is None or df_raw.empty:
            print('Falha ao carregar OpenDataSUS: df_raw is None or empty')
            return None

        colmap = {}

        for c in ['CS_SEXO','SEXO']:
            if c in df_raw.columns: colmap[c] = 'sex'
        for c in ['NU_IDADE_N','NU_IDADE']:
            if c in df_raw.columns: colmap[c] = 'age'
        for c in ['DT_SIN_PRI','DT_SINTOMA']:
            if c in df_raw.columns: colmap[c] = 'onset_date'
        for c in ['DT_NOTIFIC','DT_NOTIF']:
            if c in df_raw.columns: colmap[c] = 'report_date'
        for c in ['SG_UF_NOTA','SG_UF']:
            if c in df_raw.columns: colmap[c] = 'uf'

        df = df_raw.rename(columns=colmap).copy()
        print('Colunas mapeadas:', list(colmap.values()))

        if 'uf' in df.columns:
            uf_to_region = {
                'AC':'Norte','AP':'Norte','AM':'Norte','PA':'Norte','RO':'Norte','RR':'Norte','TO':'Norte',
                'AL':'Nordeste','BA':'Nordeste','CE':'Nordeste','MA':'Nordeste','PB':'Nordeste','PE':'Nordeste','PI':'Nordeste','RN':'Nordeste','SE':'Nordeste',
                'ES':'Sudeste','MG':'Sudeste','RJ':'Sudeste','SP':'Sudeste',
                'PR':'Sul','RS':'Sul','SC':'Sul',
                'DF':'Centro-Oeste','GO':'Centro-Oeste','MT':'Centro-Oeste','MS':'Centro-Oeste'
            }
            df['region'] = df['uf'].map(uf_to_region)

        if 'age' in df.columns:
            df['age'] = pd.to_numeric(df['age'], errors='coerce')
            print('Idade convertida para numérico.')
        if 'sex' in df.columns:
            df['sex'] = df['sex'].astype(str).str.upper().str[0].replace({'F':'F','M':'M'})
            print('Sexo convertido para str e mapeado para F/M.')
        for dcol in ['onset_date','report_date']:
            if dcol in df.columns:
                df[dcol] = pd.to_datetime(df[dcol], errors='coerce', dayfirst=True)
                print(f'Data convertida para datetime: {dcol}')

        hosp_cols = [c for c in df_raw.columns if ('HOSP' in c.upper()) or ('UTI' in c.upper())]
        if hosp_cols:
            df['hospitalized'] = np.where(df_raw[hosp_cols].notna().any(axis=1), 'internado', 'nao_internado')
        else:
            df['hospitalized'] = 'nao_internado'

        keep = [c for c in ['age','sex','region','onset_date','report_date','hospitalized'] if c in df.columns]
        if not keep:
            return None
        return df[keep].dropna(how='all')
    except Exception as e:
        print('Falha ao carregar OpenDataSUS:', e)
        return None

# Carregamento principal com fallback
df_remote = try_load_opendatasus_srag(sample_n=20000)
if df_remote is None or df_remote.empty:
    print('Usando dados SINTÉTICOS (fallback).')
    df = generate_synthetic_line_list(n=1200, seed=42)
else:
    print('Dados do OpenDataSUS (amostra) carregados.')
    df = df_remote.copy()

df.head()

## 2) Criação e **inspeção** de DataFrames

Operações úteis: `df.shape`, `df.dtypes`, `df.info()`, `df.head()`, `df.describe()`, `value_counts()`.


In [ ]:
# Inspeção rápida
print('shape:', df.shape)
display(df.dtypes)
df.info()
display(df.describe(include='all').T.head(10))

if 'sex' in df.columns:
    display(df['sex'].value_counts())

**Exercício de fixação (Seção 2)**  
Calcule o **número** e a **proporção** de valores ausentes por coluna em `df`.

In [ ]:
# Sua resposta aqui


<details><summary><strong>Solução</strong></summary>

```python
na_count = df.isna().sum()
na_prop  = df.isna().mean()
display(na_count)
display(na_prop)
```
</details>


## 3) **Seleção** de colunas e **filtragem** de linhas

- Seleção por lista: `df[['col1','col2']]`  
- Máscara booleana: `df[df['age'] >= 60]`  
- `query`: `df.query('age >= 60 and sex == "F"')`


In [ ]:
subset_cols = [c for c in ['age','sex','region','onset_date','hospitalized'] if c in df.columns]
df_sel = df[subset_cols].copy()
df_sel.head()

In [ ]:
# Exemplos
if 'age' in df_sel.columns:
    idosos = df_sel[df_sel['age'] >= 60]
    print('Nº com idade >= 60:', idosos.shape[0])
if all(c in df_sel.columns for c in ['sex','age']):
    mulheres_20a39 = df_sel.query('sex == "F" and 20 <= age <= 39')
    print('Mulheres 20–39:', mulheres_20a39.shape[0])

**Exercício de fixação (Seção 3)**  
Com `df_sel`, selecione `age`, `sex`, `region` e filtre **homens (`"M"`) com idade < 30**; mostre as 5 primeiras linhas.


In [ ]:
# Sua resposta aqui


<details><summary><strong>Solução</strong></summary>

```python
out = df_sel[['age','sex','region']].query('sex == "M" and age < 30')
out.head()
```
</details>


## 4) **Criação de colunas** com `assign` e **resumos** com `groupby().agg()`
- `assign(nova = expr)`: cria/transforma colunas.  
- `groupby(keys).agg(...)`: sumariza por grupos (contagem, média, proporção, etc.).


In [ ]:
def make_age_group(s):
    bins = [-1, 4, 11, 17, 29, 39, 49, 59, 69, 79, 200]
    labels = ['0-4','5-11','12-17','18-29','30-39','40-49','50-59','60-69','70-79','80+']
    return pd.cut(s, bins=bins, labels=labels)

if 'age' in df.columns:
    df_aug = df.assign(age_group = make_age_group(df['age']))
else:
    df_aug = df.copy()

if all(c in df_aug.columns for c in ['region','age_group']):
    resumo = (df_aug
              .groupby(['region','age_group'], dropna=False)
              .agg(casos=('age','size'))
              .reset_index()
              .sort_values(['region','age_group']))
    display(resumo.head(12))

**Exercício de fixação (Seção 4)**  
Crie `is_elderly = age >= 60` e calcule, **por região**, a **proporção** de `is_elderly` e o `n` total.


In [ ]:
# Sua resposta aqui


<details><summary><strong>Solução</strong></summary>

```python
out = (df.assign(is_elderly = df['age'] >= 60)
          .groupby('region')
          .agg(prop_elderly=('is_elderly','mean'), n=('is_elderly','size'))
          .reset_index()
          .sort_values('prop_elderly', ascending=False))
out
```
</details>


## 5) Mini‑EDA (frequências por **região** e **faixa etária**) + exemplo DATASUS
Use `value_counts()`/`groupby().size()` para obter frequências por categorias.


In [ ]:
if 'region' in df_aug.columns:
    freq_reg = df_aug['region'].value_counts(dropna=False).rename('casos').to_frame()
    display(freq_reg)
if 'age_group' in df_aug.columns:
    freq_age = df_aug['age_group'].value_counts(dropna=False).rename('casos').to_frame()
    display(freq_age)

**Exercício de fixação (Seção 5)**  
Usando `df_aug`, identifique a **região** e a **faixa etária** mais frequentes e seus totais.


In [ ]:
# Sua resposta aqui


<details><summary><strong>Solução</strong></summary>

```python
top_reg = df_aug['region'].value_counts().head(1)
top_age = df_aug['age_group'].value_counts().head(1)
display(top_reg); display(top_age)
```
</details>


## 6) Mapa mental **R ↔ Python** (foco desta aula)

| Tarefa (R/tidyverse) | `pandas` (Python) |
|---|---|
| `select(col1, col2)` | `df[['col1','col2']]` |
| `filter(x > 0 & sexo == 'F')` | `df[(df['x']>0) & (df['sexo']=='F')]` ou `df.query('x > 0 and sexo == "F"')` |
| `mutate(nova = f(x))` | `df.assign(nova = f(df['x']))` |
| `summarise(n = n()) + group_by(g)` | `df.groupby('g').agg(n=('col','size'))` |
| `arrange(desc(x))` | `df.sort_values('x', ascending=False)` |
| `rename(novo = antigo)` | `df.rename(columns={'antigo':'novo'})` |

> Joins & reshapes virão na Aula 3.


## 7) Exercícios finais (7) — dificuldade progressiva

**E1.** Selecione `age`, `sex`, `region` e conte, por `region`, **quantas linhas** existem para `sex=="F"` **e** `age >= 50` (use `query` + `groupby().size()`).

**E2.** Crie `is_adult = age >= 18` via `assign` e produza uma tabela por `region` **e** `sex` com `n` e **proporção** de adultos.

**E3.** Calcule **mediana**, **Q1** e **Q3** de `age` por `region` (use `agg` com lambdas `quantile`).

**E4.** Crie `delay_days = (report_date - onset_date).dt.days` e, por `region`, calcule a **proporção** de `delay_days > 5`.

**E5.** Crie `age_group` e calcule, por `age_group`, a **proporção** de `hospitalized=='internado'`. Liste as **três** faixas com maior proporção.

**E6.** Parâmetros em `query`: `min_age = 30`, `sexo = "M"` → `df.query('age >= @min_age and sex == @sexo')` e conte por `region`.

**E7 — Nível Olímpico (Pareto 80%)**  
Para **cada `region`**, ordene `age_group` por casos decrescentes e retorne o **menor conjunto de faixas** que acumula **≥ 80%** dos casos da região, com colunas `region`, `k_faixas_min`, `faixas_selecionadas`, `cobertura`.


In [ ]:
# E1 — sua resposta


<details><summary><strong>Solução</strong></summary>

```python
e1 = (df[['age','sex','region']]
        .query('sex == "F" and age >= 50')
        .groupby('region')
        .size().rename('n').reset_index()
        .sort_values('n', ascending=False))
e1
```
</details>


In [ ]:
# E2 — sua resposta


<details><summary><strong>Solução</strong></summary>

```python
e2 = (df.assign(is_adult = df['age'] >= 18)
        .groupby(['region','sex'])
        .agg(n=('is_adult','size'), prop_adult=('is_adult','mean'))
        .reset_index()
        .sort_values(['region','sex']))
e2
```
</details>


In [ ]:
# E3 — sua resposta


<details><summary><strong>Solução</strong></summary>

```python
e3 = (df.groupby('region')
        .agg(median=('age','median'),
             q1=('age', lambda s: s.quantile(0.25)),
             q3=('age', lambda s: s.quantile(0.75)))
        .reset_index()
        .sort_values('median', ascending=False))
e3
```
</details>


In [ ]:
# E4 — sua resposta


<details><summary><strong>Solução</strong></summary>

```python
e4 = (df.assign(delay_days = (df['report_date'] - df['onset_date']).dt.days)
        .assign(delay_gt5 = lambda d: d['delay_days'] > 5)
        .groupby('region')
        .agg(prop_delay_gt5=('delay_gt5','mean'), n=('delay_gt5','size'))
        .reset_index()
        .sort_values('prop_delay_gt5', ascending=False))
e4
```
</details>


In [ ]:
# E5 — sua resposta
min_age = 30
sexo = "M"


<details><summary><strong>Solução</strong></summary>

```python
bins = [-1, 4, 11, 17, 29, 39, 49, 59, 69, 79, 200]
labels = ['0-4','5-11','12-17','18-29','30-39','40-49','50-59','60-69','70-79','80+']
e5 = (df.assign(age_group = pd.cut(df['age'], bins=bins, labels=labels))
        .assign(is_hosp = lambda d: d['hospitalized'].eq('internado'))
        .groupby('age_group')
        .agg(prop_hosp=('is_hosp','mean'), n=('is_hosp','size'))
        .reset_index()
        .sort_values('prop_hosp', ascending=False)
        .head(3))
e5
```
</details>


In [ ]:
# E6 — sua resposta
min_age = 30
sexo = "M"


<details><summary><strong>Solução</strong></summary>

```python
min_age = 30; sexo = "M"
e6 = (df[['age','sex','region']]
        .query('age >= @min_age and sex == @sexo')
        .groupby('region')
        .size().rename('n').reset_index()
        .sort_values('n', ascending=False))
e6
```
</details>


In [ ]:
# E7 — Nível Olímpico (Pareto 80%) — sua resposta
bins = [-1, 4, 11, 17, 29, 39, 49, 59, 69, 79, 200]
labels = ['0-4','5-11','12-17','18-29','30-39','40-49','50-59','60-69','70-79','80+']


<details><summary><strong>Solução</strong></summary>

```python
bins = [-1, 4, 11, 17, 29, 39, 49, 59, 69, 79, 200]
labels = ['0-4','5-11','12-17','18-29','30-39','40-49','50-59','60-69','70-79','80+']
base = (df.assign(age_group = pd.cut(df['age'], bins=bins, labels=labels))
          .groupby(['region','age_group']).size().rename('n').reset_index())
base = base.sort_values(['region','n'], ascending=[True, False])
base['n_total_reg'] = base.groupby('region')['n'].transform('sum')
base['prop'] = base['n'] / base['n_total_reg']
base['prop_acum'] = base.groupby('region')['prop'].cumsum()

out_rows = []
for reg, sub in base.groupby('region'):
    sel = sub[sub['prop_acum'] >= 0.8]
    if sel.empty:
        k = len(sub)
        faixas = tuple(sub['age_group'].astype(str).tolist())
        cobertura = sub['prop_acum'].iloc[-1] if len(sub) else 0.0
    else:
        k_index = sel.index[0]
        k = sub.index.get_loc(k_index) + 1
        faixas = tuple(sub['age_group'].astype(str).iloc[:k].tolist())
        cobertura = float(sub['prop_acum'].iloc[k-1])
    out_rows.append({'region': reg, 'k_faixas_min': k, 'faixas_selecionadas': faixas, 'cobertura': cobertura})

e7 = pd.DataFrame(out_rows).sort_values(['k_faixas_min','region'])
e7
```
</details>


## 8) Checklist de competências
- [ ] Inspecionei `DataFrame`s com `shape`, `info`, `describe`, `value_counts`.
- [ ] Selecionei colunas e **filtrei** linhas (máscaras e `query`).
- [ ] Criei colunas com `assign` e resumi com `groupby().agg()`.
- [ ] Fiz mini‑EDA (frequências por região e faixa etária).
- [ ] Testei o exemplo de leitura **OpenDataSUS** (ou usei o fallback sintético).

### O que praticar depois
- Filtros combinados com `query` (`and`/`or`/`in`).
- Pequenas funções reutilizáveis para `assign`.
- Repetir a análise com outro ano do SIVEP ou outro recorte.

### Referências
- `pandas` docs: https://pandas.pydata.org/docs/  
- `query`: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.query.html  
- `groupby`: https://pandas.pydata.org/docs/user_guide/groupby.html  
- **OpenDataSUS – SRAG 2021–2025 (SIVEP‑Gripe)**: consulte a página oficial para links CSV e dicionário.
